In [1]:
# 단일음정 유지 능력 평가를 위한 SPICE 기반 코드

import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os

# 1. 오디오 로드
def load_audio(filepath, target_sr=16000):
    audio, sr = librosa.load(filepath, sr=target_sr)
    audio = audio / np.max(np.abs(audio))  # normalize to [-1, 1]
    return audio, sr

# 2. SPICE 모델 로드 및 피치 예측
def estimate_pitch(audio):
    # 1. hub에서 링크로 가져오기
    model = hub.load("https://tfhub.dev/google/spice/2")
    # 2. tensorflow에서 로드하기
    # model=tf.saved_model.load('model')
    output = model.signatures["serving_default"](tf.constant(audio, tf.float32))
    pitch = output["pitch"].numpy().flatten()
    confidence = (1 - output["uncertainty"]).numpy().flatten()
    return pitch, confidence

# 3. 유효한 피치 (confidence >= 0.9)만 필터링
def filter_pitch(pitch, confidence, threshold=0.9):
    return [p if c >= threshold else 0 for p, c in zip(pitch, confidence)]

# 4. 이동 표준편차 계산
def moving_std(seq, win=5):
    padded = np.pad(seq, (win//2,), mode='edge')
    return [np.std(padded[i:i+win]) for i in range(len(seq))]

# 5. 단일음정 구간 판별 및 평가
def evaluate_pitch_stability(filtered_pitch, std_threshold=1.5, fps=100):
    pitch_std = moving_std(filtered_pitch, win=5)
    mono_flags = [s < std_threshold and p > 0 for s, p in zip(pitch_std, filtered_pitch)]
    mono_duration = sum(mono_flags) / fps  # 초 단위
    total_duration = len(filtered_pitch) / fps
    stable_ratio = mono_duration / total_duration if total_duration > 0 else 0
    return stable_ratio, mono_duration, total_duration, mono_flags

# 6. 전체 파이프라인 함수
def analyze_pitch_stability(filepath):
    audio, sr = load_audio(filepath)
    pitch, confidence = estimate_pitch(audio)
    filtered_pitch = filter_pitch(pitch, confidence)
    stable_ratio, mono_duration, total_duration, mono_flags = evaluate_pitch_stability(filtered_pitch)

    print(f"총 길이: {total_duration:.2f}s")
    print(f"단일음정 유지 시간: {mono_duration:.2f}s")
    print(f"안정성 비율: {stable_ratio:.2%}")

    # 시각화
    plt.figure(figsize=(12, 4))
    plt.plot(filtered_pitch, label='Filtered Pitch (Hz)')
    plt.plot(moving_std(filtered_pitch), label='Moving Std')
    plt.axhline(y=1.5, color='r', linestyle='--', label='Threshold')
    plt.legend()
    plt.title("Pitch Stability Analysis")
    plt.show()

    return stable_ratio

print("함수들이 정의되었습니다.")

함수들이 정의되었습니다.


In [3]:
# 오디오 파일 분석 실행
os.chdir("/Users/kimberlyjojohirn/Downloads/SeSAC/project/spice")
filepath = "0_아/e_1_1.wav"

# 파일 존재 확인
if os.path.exists(filepath):
    print(f"파일을 찾았습니다: {filepath}")
    result = analyze_pitch_stability(filepath)
    print(f"최종 안정성 점수: {result:.2%}")
else:
    print(f"파일을 찾을 수 없습니다: {filepath}")
    print("현재 디렉토리:", os.getcwd())
    print("spice 폴더 내용:")
    if os.path.exists("project/spice"):
        for item in os.listdir("project/spice"):
            print(f"  {item}")
    else:
        print("project/spice 폴더가 존재하지 않습니다.")

파일을 찾았습니다: 0_아/e_1_1.wav


2025-08-02 04:52:09.348953: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-08-02 04:52:09.349182: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-08-02 04:52:09.349194: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-08-02 04:52:09.349888: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-02 04:52:09.350303: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


2025-08-02 04:52:09.431687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2025-08-02 04:52:09.504358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2025-08-02 04:52:09.700190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-08-02 04:52:09.837174: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 5791708144260403484
2025-08-02 04:52:09.837187: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3466542408690556931
2025-08-02 04:52:09.837190: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8873224018497675573
2025-08-02 04:52:09.837194: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 13750489192917500577
2025-08-02 04:52:09.837197: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8255454483872547017
2025-08-02 04:52:09.837200: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous r

FailedPreconditionError: Graph execution error:

Detected at node 'encoder/dense/bias/read' defined at (most recent call last):
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/Users/kimberlyjojohirn/opt/anaconda3/envs/SeSAC/lib/python3.9/site-p